# Agenci Azure AI z obsługą Model Context Protocol (MCP)

Ten notatnik pokazuje, jak korzystać z agentów Azure AI z narzędziami Model Context Protocol (MCP), aby stworzyć inteligentnego agenta, który może wykorzystywać zewnętrzne serwery MCP do rozszerzonych możliwości.


## Instalacja wymaganych pakietów NuGet

Najpierw musimy zainstalować pakiet Azure AI Agents Persistent, który dostarcza podstawową funkcjonalność do pracy z Azure AI Agents.


## Korzyści z uwierzytelniania bez kluczy

Ten notebook demonstruje **uwierzytelnianie bez kluczy**, które oferuje kilka zalet:
- ✅ **Brak konieczności zarządzania kluczami API** - Wykorzystuje uwierzytelnianie oparte na tożsamości Azure
- ✅ **Zwiększone bezpieczeństwo** - Brak przechowywania sekretów w kodzie lub konfiguracji
- ✅ **Automatyczna rotacja poświadczeń** - Azure zarządza cyklem życia poświadczeń
- ✅ **Dostęp oparty na rolach** - Wykorzystuje Azure RBAC do precyzyjnego zarządzania uprawnieniami

`DefaultAzureCredential` automatycznie wybiera najlepsze dostępne źródło poświadczeń:
1. Zarządzana tożsamość (podczas uruchamiania w Azure)
2. Poświadczenia Azure CLI (w trakcie rozwoju)
3. Poświadczenia Visual Studio
4. Zmienne środowiskowe (jeśli skonfigurowane)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Zainstaluj pakiet Azure Identity do uwierzytelniania za pomocą usług Azure przy użyciu DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importuj wymagane przestrzenie nazw

Zaimportuj niezbędne przestrzenie nazw dla Azure AI Agents i Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfiguracja klienta Azure AI Agent (uwierzytelnianie bez klucza)

Skonfiguruj zmienne i utwórz PersistentAgentsClient, korzystając z **uwierzytelniania bez klucza**:
- **projectEndpoint**: Punkt końcowy projektu Azure AI Foundry
- **modelDeploymentName**: Nazwa wdrożonego modelu AI (GPT-4.1 nano)
- **mcpServerUrl**: URL serwera MCP (Microsoft Learn API)
- **mcpServerLabel**: Etykieta identyfikująca serwer MCP
- **DefaultAzureCredential**: Wykorzystuje zarządzaną tożsamość, Azure CLI lub inne źródła poświadczeń (bez potrzeby używania kluczy API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Utwórz definicję narzędzia MCP

Utwórz definicję narzędzia MCP, które łączy się z serwerem Microsoft Learn MCP. Dzięki temu agent będzie miał dostęp do treści i dokumentacji Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Utwórz agenta AI

Utwórz trwałego agenta AI z określonym modelem i narzędziami MCP. Agent jest skonfigurowany z:
- Modelem GPT-4.1 nano
- Instrukcjami dotyczącymi korzystania z narzędzi MCP jako wsparcia
- Dostępem do serwera Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Utwórz wątek i wyślij wiadomość

Utwórz wątek rozmowy i wyślij wiadomość użytkownika z pytaniem o różnicę między Azure OpenAI a OpenAI. To sprawdzi zdolność agenta do korzystania z narzędzi MCP w celu dostarczenia dokładnych informacji.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurowanie zasobów narzędzia MCP (bez klucza)

Skonfiguruj zasoby narzędzia MCP. Aby uzyskać w pełni bezkluczowe podejście, możesz usunąć niestandardowe nagłówki, jeśli serwer MCP obsługuje uwierzytelnianie oparte na tożsamości Azure. Poniższy przykład pokazuje, jak dodać nagłówki w razie potrzeby, ale w scenariuszach bezkluczowych mogą one nie być wymagane.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Rozpocznij Uruchomienie Agenta

Utwórz i rozpocznij proces, aby przetworzyć wiadomość użytkownika. Agent skorzysta z skonfigurowanych narzędzi i zasobów MCP, aby wygenerować odpowiedź.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitorowanie uruchomienia i obsługa zatwierdzeń narzędzi (bez kluczy)

Monitoruj status uruchomienia agenta i obsługuj wymagane zatwierdzenia narzędzi. Ten proces:
1. Czeka na zakończenie uruchomienia lub wymaga podjęcia działania
2. Automatycznie zatwierdza wywołania narzędzi MCP, gdy jest to konieczne
3. W przypadku uwierzytelniania bez kluczy, nagłówki mogą nie być potrzebne, jeśli serwer MCP obsługuje tożsamość Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Wyświetlanie wyników rozmowy

Pobierz i wyświetl wszystkie wiadomości w wątku, pokazując zarówno pytanie użytkownika, jak i odpowiedź agenta. Wiadomości są wyświetlane w porządku chronologicznym z oznaczeniami czasowymi i wskaźnikami ról.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
